In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import requests
import wikipedia
import re
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen
from scrapy import selector
import datetime as dt
import pickle
from skimage import io
from IPython.display import clear_output
import pdb
plt.style.use('ggplot')
%matplotlib inline

# Obtain Scoresheet

In [176]:
def get_squad(batting_sheet):
    """Gets team members, captain and keeper
    """
    team = []
    
    # batsmen who batted
    batsmen = batting_sheet.find_all('div', class_ = "wrap batsmen")
    for bat in batsmen:
        player = bat.find('div', class_ = "cell batsmen").text
        if player.find('(c)')>-1:
            captain = player.replace('(c)','').replace('†','').strip().lower()
        if player.find('†')>-1:
            keeper = player.replace('†','').replace('(c)','').strip().lower()
        player = player.replace('†','').replace('(c)','').strip()
        team.append(player.lower())
    did_not_bat = batting_sheet.find('div', class_ = "wrap dnb")
    
    
    # Now batsmen who did not bat
    for dnb in did_not_bat.find_all('a'):
        team.append(dnb.text.lower())
    return team, captain, keeper


def get_full_name(name, team):
    """Accepts last name and returns players name
    """
    name = name.lower()
    name = name.replace('†','').replace('(c)','')
    for member in team:
        if member.lower().find(name) > -1:
            return member
        
def get_innings(innings, teams):
    """Returns batting and bowling innings
    """
    
    for i,inning in enumerate(innings):
        for team in teams:
            if inning.find(team)>-1:
                innings[i] = team
    
    return innings
                

        
def get_toss_outcome(toss_string):
    pass

In [225]:

outs = ['b', 'c', 'lbw', 'st', 'hw', 'ro', 'hand', 'obs']

def how_out(dismissal, team):
    """Returns triplet tuple as (how, fielder, bowler)
    """
    
    dismissal = dismissal.lower().strip()
    
    if dismissal == 'not out':
        return 'not out', '', ''
    
    ## No fielder involved
    # run out
    if dismissal.find('run out')>-1:
        how = 'run out'
        fielder = ''
        bowler = ''
        return how, fielder, bowler
    
    # handled
    if dismissal.find('handled')>-1:
        how = 'handled the ball'
        fielder = ''
        bowler = ''
        return how, fielder, bowler

    # obstructing the fieldsman
    if dismissal.find('obstruct') > -1:
        how = 'obstruct'
        fielder = ''
        blower = ''
        return how, fielder, bowler
    
    bowler = re.findall(r"b\s(.+)", dismissal)
    fielder = re.findall(r"c\s(.+)\sb", dismissal)
    
    if len(bowler)>0:
        bowler = bowler[0]
    if len(fielder)>0:
        fielder = fielder[0]
    
    #print(bowler, fielder)
    
    if len(fielder) == 0:
        keeper = re.findall(r"st\s†(.+)\sb", dismissal)
        if len(keeper)>0:
            keeper = keeper[0]
    
    # bowled
    if len(fielder) == 0 and dismissal == 'b ' + bowler:
        how = 'bowled'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # caught
    if len(fielder) > 0:
        how = 'caught'
        fielder = get_full_name(fielder, team)
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # lbw
    if dismissal.find('lbw')>-1 or dismissal.find('leg before')>-1:
        how = 'lbw'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # hit wicket
    if dismissal.find('hit wicket')>-1:
        how = 'hit wicket'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    
    # caught and bowled
    if dismissal.find('&') > -1:
        how = 'caught'
        bowler = get_full_name(bowler, team)
        fielder = bowler
        return how, fielder, bowler
    
    
    # Stumped
    if dismissal.find('st †') > -1:
        how = 'stumped'
        fielder = get_full_name(keeper, team)
        bowler = get_full_name(bowler, team)
        return 'stumped',fielder, bowler


In [226]:
# how_out('st †Kaluwitharana b Muralitharan', squad['sri lanka'])

In [227]:
# def get_scoresheet(link, format):
match = pickle.load(open('test_list', "rb" ))
year = 1932
num = 7
match[year].reset_index(inplace=True)

link = match[year].iloc[num,-1]
print(link)
matchID = match[year].iloc[num,0]

if matchID.find('odi')>-1:
    match_format = 'odi'
elif matchID.find('test')>-1:
    match_format = 'test'

team_1 = match[year].iloc[num,:].team_1
team_2 = match[year].iloc[num,:].team_2
team_names = [team_1, team_2]
opponents = {}
opponents[team_1] = team_2
opponents[team_2] = team_1

# Get the link
r = requests.get(link)
soup = BeautifulSoup(r.content,'lxml')

# Number of score sheets = number of innings
# Each scoresheet contains both batting and bowling information
scoresheets = soup.find_all('article', class_ = "sub-module scorecard")


match_details = soup.find('div', class_ = "match-detail-container")

##############################################################################################
############################# .   Get Game Info   ############################################

MATCH_DETAILS = {}
MATCH_DETAILS['stadium'] = match_details.find('div', class_ = "stadium-details").text.lower().strip()
for detail in match_details.find_all('div', class_ = "match-detail--item"):
    left = detail.find('div', class_ = "match-detail--left").text.lower().strip()
    MATCH_DETAILS[left] = []
    for r in detail.find('div', class_ = "match-detail--right").find_all('span'):
        MATCH_DETAILS[left].append(r.text.lower().strip())
MATCH_DETAILS






##############################################################################################
############################      Get Team Info    ###########################################

num_innings = len(scoresheets)

bat_innings = []
ball_innings = []

squad = {}
squad[team_1] = []
squad[team_2] = []

captain = {}
captain[team_1] = []
captain[team_2] = []

keeper = {}
keeper[team_1] = []
keeper[team_2] = []

# Get innings Info, team, captain, keeper
for i,sheet in enumerate(scoresheets):
    
    
    if match_format == 'odi':
        my_str = sheet.find('div', class_ = "accordion-header").text.lower().strip()
        this_innings = re.findall(r'(.+)\sinnings', my_str)[0]
    elif match_format == 'test':
        my_str = sheet.find('div', class_ = "accordion-header").text.lower().strip()
        this_innings = re.findall(r'(.+)\s\d', my_str)[0]

    bat_innings.append(this_innings)

ball_innings = [opponents[x] for x in bat_innings]

# Get squad for team batting first
team_batting_first = bat_innings[0]
batting_sheet = scoresheets[0].find("div", class_="scorecard-section batsmen")
squad[team_batting_first], captain[team_batting_first], keeper[team_batting_first] = get_squad(batting_sheet)

# Get squad for team batting second
team_batting_second = bat_innings[1]
batting_sheet = scoresheets[1].find("div", class_="scorecard-section batsmen")
squad[team_batting_second], captain[team_batting_second], keeper[team_batting_second] = get_squad(batting_sheet)


TEAM_INFO = {}
TEAM_INFO['num_innings'] = num_innings

TEAM_INFO['bat_innings'] = bat_innings
TEAM_INFO['ball_innings'] = ball_innings

TEAM_INFO['team_batting_first'] = team_batting_first
TEAM_INFO['team_batting_second'] = team_batting_second

TEAM_INFO['squad'] = squad
TEAM_INFO['captain'] = captain
TEAM_INFO['keeper'] = keeper




#############################################################################################
####################    Get Scoresheets/ Batting and Bowling        #########################
#############################################################################################

    
# Now get batting and bowling info
BATTING = {}
BOWLING = {}
for s,sheet in enumerate(scoresheets):
    
    batting_sheet = sheet.find("div", class_="scorecard-section batsmen")
    bowling_sheet = sheet.find("div", class_ = "scorecard-section bowling")
    inning = 'inning_' + str(s+1)
    print("In:", inning)
    
    batting_team = bat_innings[s]
    bowling_team = ball_innings[s]
    
    ## DEFINE HEADERS
    # get all the header.
    # Some games have SR and balles, some not, so important to get it on a game-by-game basis
    all_headers = []
    for header in batting_sheet.find('div', class_ = "wrap header").find_all('div'):
        all_headers.append(header.text.lower())
        
    # The commentary header is usually emptty: just add commentary
    if len(all_headers[1])==0:
        all_headers[1] = 'commentary'
        
    # Create dataframe
    all_headers =  all_headers + ['how', 'fielder', 'bowler'] + ['matchID', 'year', 'team', 'innings']
    score_card_batting = pd.DataFrame(columns = all_headers)
    
    # Iterate over batesmen
    all_batsmen  = batting_sheet.find_all('div', class_ = "wrap batsmen")
    for i, batsman in enumerate(all_batsmen):
        
        # iterate over cells
        for j,info in enumerate(batsman.find_all('div')):
            # print(j,info.text, type(info.text))
        
            if j == 0: #name
                player = info.text.lower().replace('(c)','').replace('†','').strip()
                score_card_batting.loc[i, all_headers[j]] = player
                continue
            if j == 1: #commentary
                # print(info.text.lower().strip())
                how,fielder,bowler = how_out(info.text.lower().strip(), squad[bowling_team])
                        
            
            score_card_batting.loc[i, 'how'] = how
            score_card_batting.loc[i, 'fielder'] = fielder
            score_card_batting.loc[i, 'bowler'] = bowler
            score_card_batting.loc[i, all_headers[j]] = info.text.lower().strip()
        
    did_not_bat = set(squad[batting_team]) - set(score_card_batting.batsmen.values)
    #print(did_not_bat)
    
    l = len(score_card_batting)
    for dnb in did_not_bat:
        score_card_batting.loc[l, all_headers[0]] = dnb
        l = l+1
    
    # Now get extras
    extras = sheet.find('div', class_ = "wrap extras").find_all('div')[1].text
    l = len(score_card_batting)
    score_card_batting.loc[l, all_headers[0]] = 'extras'
    score_card_batting.loc[l, all_headers[1]] =  extras
    
    # Total
    total = sheet.find('div', class_ = "wrap total").find_all('div')[1].text
    l = len(score_card_batting)
    score_card_batting.loc[l, all_headers[0]] = 'total'
    score_card_batting.loc[l, all_headers[1]] =  total

    
            
            
    # Now fill match-cells: common to all batsmen
    score_card_batting.loc[:, 'matchID'] = matchID
    score_card_batting.loc[:, 'year'] = year
    score_card_batting.loc[:, 'innings'] = s+1
    score_card_batting.loc[:, 'team'] = batting_team
    
    BATTING[inning] = score_card_batting.fillna(0)
            

#     ############## Bowling Info
    bowling_sheet = sheet.find("div", class_ = "scorecard-section bowling")
    bowling_headers = []
    for header in bowling_sheet.find('thead').find_all('th'):
        bowling_headers.append(header.text.lower().strip())
        
    bowling_headers = bowling_headers +  ['matchID', 'year', 'team', 'innings']
    score_card_bowling = pd.DataFrame(columns = bowling_headers)  
    
    # bowlers who bowled
    for b,bowler in enumerate(bowling_sheet.find('tbody').find_all('tr')): 
        for c,cell in enumerate(bowler.find_all('td')):
            score_card_bowling.loc[b,bowling_headers[c]]= cell.text.lower().strip()
    # bowlers who did not bowl
    dnb = set(squad[bowling_team]) - set(score_card_bowling[bowling_headers[0]].values)
    l = len(score_card_bowling)
    for d in dnb:
        score_card_bowling.loc[l,bowling_headers[0]] = d
        l = l+1
    
    # Now fill match-cells: common to all batsmen
    score_card_bowling.loc[:, 'matchID'] = matchID
    score_card_bowling.loc[:, 'year'] = year
    score_card_bowling.loc[:, 'innings'] = s+1
    score_card_bowling.loc[:, 'team'] = bowling_team
    
    BOWLING[inning] = score_card_bowling.fillna(0)

        
    

http://stats.espncricinfo.com//ci/engine/match/62607.html
In: inning_1
In: inning_2
In: inning_3
In: inning_4


In [220]:
TEAM_INFO

{'num_innings': 4,
 'bat_innings': ['australia', 'england', 'australia', 'england'],
 'ball_innings': ['england', 'australia', 'england', 'australia'],
 'team_batting_first': 'australia',
 'team_batting_second': 'england',
 'squad': {'australia': ['jhw fingleton',
   'wm woodfull',
   "lpj o'brien",
   'dg bradman',
   'sj mccabe',
   'vy richardson',
   'was oldfield',
   'cv grimmett',
   'tw wall',
   "wj o'reilly",
   'h ironmonger'],
  'england': ['h sutcliffe',
   'res wyatt',
   'wr hammond',
   'nawab of pataudisnr',
   'm leyland',
   'dr jardine',
   'leg ames',
   'gob allen',
   'h larwood',
   'w voce',
   'we bowes']},
 'captain': {'australia': 'wm woodfull', 'england': 'dr jardine'},
 'keeper': {'australia': 'was oldfield', 'england': 'leg ames'}}

In [221]:
BOWLING['inning_2']

,bowling,,o,m,r,w,econ,wd,nb,,matchID,year,team,innings
0,tw wall,,21,4,52,4,2.47,0,0,,test # 221,1932,australia,2
1,wj o'reilly,,34.3,17,63,5,1.82,0,0,,test # 221,1932,australia,2
2,cv grimmett,,16,4,21,1,1.31,0,0,,test # 221,1932,australia,2
3,h ironmonger,,14,4,28,0,2.00,0,0,,test # 221,1932,australia,2
4,dg bradman,0,0,0,0,0,0,0,0,0,test # 221,1932,australia,2
5,was oldfield,0,0,0,0,0,0,0,0,0,test # 221,1932,australia,2
6,vy richardson,0,0,0,0,0,0,0,0,0,test # 221,1932,australia,2
7,sj mccabe,0,0,0,0,0,0,0,0,0,test # 221,1932,australia,2
8,jhw fingleton,0,0,0,0,0,0,0,0,0,test # 221,1932,australia,2
9,lpj o'brien,0,0,0,0,0,0,0,0,0,test # 221,1932,australia,2


In [222]:
BATTING['inning_4']

,batsmen,commentary,r,b,m,4s,6s,sr,,how,fielder,bowler,matchID,year,team,innings
0,h sutcliffe,b o'reilly,33,63,53,4,0,52.38,,bowled,,wj o'reilly,test # 221,1932,england,4
1,m leyland,b wall,19,51,58,2,0,37.25,,bowled,,tw wall,test # 221,1932,england,4
2,nawab of pataudisnr,c fingleton b ironmonger,5,36,34,0,0,13.88,,caught,jhw fingleton,h ironmonger,test # 221,1932,england,4
3,wr hammond,c o'brien b o'reilly,23,43,51,2,0,53.48,,caught,lpj o'brien,wj o'reilly,test # 221,1932,england,4
4,dr jardine,c mccabe b ironmonger,0,3,2,0,0,0.00,,caught,sj mccabe,h ironmonger,test # 221,1932,england,4
5,leg ames,c fingleton b o'reilly,2,10,8,0,0,20.00,,caught,jhw fingleton,wj o'reilly,test # 221,1932,england,4
6,res wyatt,lbw b o'reilly,25,50,54,3,0,50.00,,lbw,,wj o'reilly,test # 221,1932,england,4
7,gob allen,st †oldfield b ironmonger,23,63,51,1,0,36.50,,stumped,was oldfield,h ironmonger,test # 221,1932,england,4
8,h larwood,c wall b ironmonger,4,7,11,0,0,57.14,,caught,tw wall,h ironmonger,test # 221,1932,england,4
9,w voce,c o'brien b o'reilly,0,5,3,0,0,0.00,,caught,lpj o'brien,wj o'reilly,test # 221,1932,england,4


In [218]:
MATCH_DETAILS

{'stadium': 'melbourne cricket ground',
 'series': ['england [marylebone cricket club] tour of australia 1932/33'],
 'toss': ['australia , elected to bat first'],
 'series result': ['5-match series level 1-1'],
 'match number': ['test no. 221'],
 'season': ['1932/33'],
 'match days': ['30,31 december 1932, 2,3 january 1933 (timeless match)'],
 'test debut': ["leo o'brien"],
 'umpires': ['george borwick', 'george hele']}